In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=service, options=options)

# Ganti dengan URL halaman review toko di Tokopedia
url = "https://play.google.com/store/apps/details?id=com.tokopedia.tkpd&hl=en"

driver.get(url)

reviews = []
count = 1

try:
    review_button = driver.find_element(By.CSS_SELECTOR, 'button[ class="VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-dgl2Hf ksBjEc lKxP2d LQeN7 aLey0c" ]')
    review_button.click()
    time.sleep(2)
    print("clicking")
    
    scroll_count = 0
    while scroll_count < 30 :
        scroll = driver.find_element(By.CLASS_NAME, 'odk6He')
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'end'});", scroll)
        time.sleep(2)
        scroll_count = scroll_count + 1
except:
    print("There is nothing")

while count < 100:
    # Ambil elemen review
    time.sleep(3)  # Tunggu halaman masuk
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try:
        review_elements = soup.findAll("div", attrs={'class':'RHo1pe'})

        for review in review_elements:
            username = review.find("div", attrs={"class":"X5PpBb"}).text
            comment = review.find("div", attrs={"class":"h3YV2d"}).text
            
            reviews.append({
                "username": username,
                "review": comment
            })
            print(count)
            count=count + 1
            
    except:
        print("there is error")


driver.quit()

# Simpan ke CSV
df = pd.DataFrame(reviews)
df.to_csv("tokopedia_review.csv", index=False)
print("Scraping selesai! Data disimpan dalam tokopedia_reviews.csv")

clicking
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pandas as pd

df = pd.read_csv("tokopedia_review.csv")  # Load your CSV fil

# Load SpaCy's English model
nlp = spacy.load("en_core_web_sm")

# Preprocessing function using SpaCy
def preprocess_text_spacy(text):
    # Convert to lowercase and process the text with SpaCy
    doc = nlp(text.lower())
    
    # Remove stopwords, punctuation, and non-alphabetic tokens, then lemmatize
    filtered_tokens = [
        token.lemma_ for token in doc if token.is_alpha and token.text not in STOP_WORDS
    ]
    
    # Return a space-separated string of lemmatized tokens
    return " ".join(filtered_tokens)

# Apply preprocessing to all reviews
df["processed_review"] = df["review"].apply(preprocess_text_spacy)

# Save preprocessed data
df.to_csv("preprocessed_reviews_spacy.csv", index=False)
print("Preprocessing complete! Data saved as preprocessed_reviews_spacy.csv")

Preprocessing complete! Data saved as preprocessed_reviews_spacy.csv


In [5]:
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

df["entities"] = df["review"].astype(str).apply(extract_entities)
print(df.head())

df.to_csv("entities_review.csv", index=False)
print("Preprocessing complete! Data saved as entities_review.csv")

         username                                             review  \
0            Yoga  Bro, fix the chat feature in this app. Also, s...   
1               D  Tokopedia used to be my favorite marketplace t...   
2  Daniel Chandra  They're faking the price by displaying the dis...   
3        Lou Park  Horrifyingly inefficient user interface with u...   
4       dibahusna  Everything went well when it did and went as a...   

                                    processed_review  \
0  bro fix chat feature app stop display discount...   
1  tokopedia favorite marketplace need love catch...   
2  fake price display discount price voucher upfr...   
3  horrifyingly inefficient user interface unreli...   
4  go go awful think center wait connect customer...   

                                            entities  
0                                        [(UI, ORG)]  
1  [(Tokopedia, GPE), (UI, ORG), (400pcs, CARDINA...  
2  [(only one, CARDINAL), (more than one, CARDINA...  
3         

In [7]:
import networkx as nx
from pyvis.network import Network

G = nx.Graph()

for index, row in df.iterrows():
    user = row["username"]
    review = row["processed_review"]
    entities = row["entities"]

    # Tambahkan node user
    G.add_node(user, type="User")

    # Tambahkan node entitas (Produk/Sentimen)
    for ent_text, ent_label in entities:
        G.add_node(ent_text, type=ent_label)
        G.add_edge(user, ent_text, relation="review")  # Hubungkan user dengan entitas

# Visualisasi Knowledge Graph
net = Network(notebook=True, height="800px", width="100%", bgcolor="#222222", font_color="white")
net.from_nx(G)

# Tambahkan fitur interaktif
net.toggle_physics(True)  # Aktifkan fisika agar node tidak tumpang tindih
net.force_atlas_2based(gravity=-50, central_gravity=0.01)  # Sesuaikan gaya tarik antar node

# Simpan ke file HTML dan tampilkan
net.show("knowledge_graph.html")

knowledge_graph.html


In [28]:
# Convert nodes and edges into sentences
def graph_to_text(graph):
    text_data = []
    for node in graph.nodes():
        relations = [f"{node} is related to {nbr} via {graph[node][nbr]['relation']}"
                     for nbr in graph.neighbors(node)]
        text_data.extend(relations)
    return text_data

text_corpus = graph_to_text(G)
print(text_corpus)

['ANDRI is related to aman via review', 'ANDRI is related to engga bocor kirim cepat via review', 'aman is related to ANDRI via review', 'aman is related to Vica via review', 'aman is related to Raflie via review', 'aman is related to S***l via review', 'engga bocor kirim cepat is related to ANDRI via review', 'K***s is related to thank you via review', 'K***s is related to asli kirim via review', 'thank you is related to K***s via review', 'asli kirim is related to K***s via review', 'Marissa is related to sesuai pesan via review', 'sesuai pesan is related to Marissa via review', 'sesuai pesan is related to Zulham via review', 'C***y is related to benar-benar via review', 'C***y is related to gak via review', 'benar-benar is related to C***y via review', 'gak is related to C***y via review', 'd***f is related to wangi via review', 'wangi is related to d***f via review', 'wangi is related to Sulthan via review', 'wangi is related to S***a via review', 'wangi is related to R***n via rev

In [29]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Convert text corpus to embeddings
embeddings = embedder.encode(text_corpus)
embeddings = np.array(embeddings).astype("float32")

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance (cosine similarity)
index.add(embeddings)

print(f"FAISS Index contains {index.ntotal} vectors")

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run P

FAISS Index contains 632 vectors


In [35]:
def retrieve_knowledge(query, top_k=3):
    query_embedding = embedder.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    
    retrieved_info = [text_corpus[idx] for idx in indices[0] if idx < len(text_corpus)]
    return retrieved_info

query = "Review bagus"
retrieved = retrieve_knowledge(query)
print("Retrieved Knowledge:", retrieved)

Retrieved Knowledge: ['bagus is related to S***i via review', 'S***i is related to bagus via review', 'bagus is related to Bayu via review']
